In [2]:
import torch.nn as nn
import torch

from torchtext.vocab import GloVe

class GloveModel(nn.Module):
    def __init__(self, embed_dim = 200):
        super(GloveModel, self).__init__()
        global_vectors = GloVe(name='6B', dim=embed_dim)
        glove_weights = torch.load(f".vector_cache/glove.6B.{embed_dim}d.txt.pt")
        self.embedding = nn.Embedding.from_pretrained(glove_weights[2])

    def forward(self, x):
        x = self.embedding(x)
        return x

/Users/omarahmed/miniconda3/envs/main/lib/python3.12/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/omarahmed/miniconda3/envs/main/lib/python3.12/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import GloVe


tokenizer = get_tokenizer("basic_english")

def tokenize(text):
    """Tokenizes a string into a list of words."""
    return tokenizer(text)


def detokenize(tokens):
    """Detokenizes a list of words into a string."""
    return ' '.join(tokens)


tokenize("Hello, world!")  # Example usage


['hello', ',', 'world', '!']

In [60]:
glove = GloVe(name='6B', dim=300).get_vecs_by_tokens(tokenize("[sos] Hello, world []!"))
print(glove)

#  decoder the indices
def decode_indices(indices):
    """Decodes a list of indices into a string."""
    return ' '.join([glove.itos[i] for i in indices])

NameError: name 'tokenize' is not defined

In [48]:
text = "Testing the GloVe [EOS] model."
tokens = tokenize(text)
print("Tokens:", tokens)

indices = [glove.stoi[token] for token in tokens]
print("Indices:", indices)

Tokens: ['testing', 'the', 'glove', '[eos]', 'model', '.']


AttributeError: 'Tensor' object has no attribute 'stoi'

In [40]:
glove.vectors[torch.tensor([400001])]

IndexError: index 400001 is out of bounds for dimension 0 with size 400000

In [ ]:
from torchtext.vocab import GloVe, vocab
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
import torch
import torch.nn as nn

#define your model that accepts pretrained embeddings 
class TextClassificationModel(nn.Module):

    def __init__(self, pretrained_embeddings, num_class, freeze_embeddings = False):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag.from_pretrained(pretrained_embeddings, freeze = freeze_embeddings, sparse=True)
        self.fc = nn.Linear(pretrained_embeddings.shape[1], num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

train_iter = AG_NEWS(split = 'train')
num_class = len(set([label for (label, _) in train_iter]))
unk_token = "<unk>"
unk_index = 0
glove_vectors = GloVe()
glove_vocab = vocab(glove_vectors.stoi)
glove_vocab.insert_token("<unk>",unk_index)
#this is necessary otherwise it will throw runtime error if OOV token is queried 
glove_vocab.set_default_index(unk_index)
pretrained_embeddings = glove_vectors.vectors
pretrained_embeddings = torch.cat((torch.zeros(1,pretrained_embeddings.shape[1]),pretrained_embeddings))

#instantiate model with pre-trained glove vectors
glove_model = TextClassificationModel(pretrained_embeddings, num_class)


tokenizer = get_tokenizer("basic_english")
train_iter = AG_NEWS(split = 'train')
example_text = next(train_iter)[1]
tokens = tokenizer(example_text)
indices = glove_vocab(tokens)
text_input = torch.tensor(indices)
offset_input = torch.tensor([0])

model_output = glove_model(text_input, offset_input)


In [ ]:
from torchtext.vocab import GloVe, vocab
glove = GloVe(name='6B', dim=300)
glove_vocab = vocab(glove.stoi)
special_tokens = ["[unk]", "[pad]", "[mask]", "[sos]", "[eos]", "[sep]"]
for i, token in enumerate(special_tokens):
    glove_vocab.insert_token(token, i)
    glove_vocab.set_default_index(i)
pretrained_embeddings = glove.vectors
pretrained_embeddings = torch.cat(
    (torch.zeros(len(special_tokens),
                 pretrained_embeddings.shape[1]), pretrained_embeddings))

#  test the vectors
print("Pretrained Embeddings Shape:", pretrained_embeddings.shape)
#  test the vocab
print("Vocab Size:", len(glove_vocab))
#  test the special tokens
for token in special_tokens:
    print(f"{token}: {glove_vocab[token]}")


text = "Testing the GloVe [EOS] model."
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer(text)


Pretrained Embeddings Shape: torch.Size([400006, 300])
Vocab Size: 400005
[unk]: 0
[pad]: 1
[mask]: 2
[sos]: 3
[eos]: 4
[sep]: 5


NameError: name 'detokenize' is not defined

In [77]:
embedding_layer = nn.Embedding.from_pretrained(pretrained_embeddings)
#  test the embedding layer with a sample input
sample_input = torch.tensor([glove_vocab["[eos]"], glove_vocab["[unk]"]])
embedded_output = embedding_layer(sample_input)
print("Embedded Output Shape:", embedded_output[1])

Embedded Output Shape: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0

In [79]:
pretrained_embeddings[0:7]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539]])

In [ ]:
    tokenizer = get_tokenizer("basic_english")
    

In [102]:
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import GloVe, vocab
import torch
torchtext.disable_torchtext_deprecation_warning()


class gloveTokenizer:

    def __init__(
        self,
        ver: str = "6B",
        dim: int = 300,
        special_tokens: list[str] = [
            "[UNK]", "[PAD]", "[MASK]", "[SOS]", "[EOS]", "[SEP]"
        ],
        max_length: int = None,
    ):
        """
        Initialize the GloVe tokenizer.

        Args:
            ver (str): The name of the GloVe model to use. Default is "6B".
            dim (int): The dimensionality of the GloVe vectors. Default is 300.
            special_tokens (list[str]): A list of special tokens to add to the vocabulary.
            max_length (int): The maximum length for the tokenizer. Default is None.
        """

        glove = GloVe(name=ver, dim=dim)
        glove_vocab = vocab(glove.stoi)
        for i, token in enumerate(special_tokens):
            glove_vocab.insert_token(token, i)
        glove_vocab.set_default_index(glove_vocab["[UNK]"])
        pretrained_embeddings = glove.vectors
        pretrained_embeddings = torch.cat(
            (torch.randn(len(special_tokens), pretrained_embeddings.shape[1]),
             pretrained_embeddings))

        self.special_tokens = special_tokens
        self.pretrained_embeddings = pretrained_embeddings
        self.tokenizer = get_tokenizer("basic_english")
        self.glove_vocab = glove_vocab
        self.max_length = max_length
        self.pad_idx = glove_vocab["[PAD]"]

    def set_max_length(self, max_length: int):
        """
        Set the maximum length for the tokenizer.

        Args:
            max_length (int): The maximum length for the tokenizer.
        """
        self.max_length = max_length

    def tokenize(self, text1: str, text2: str = None) -> list[str]:
        """
        Tokenize one or two input strings, add special tokens, and PAD to max_length if set.

        Args:
            text1 (str): The first input string.
            text2 (str, optional): The second input string (for paired input).

        Returns:
            list[str]: The tokenized and padded list with special tokens.
        """
        tokens = ["[SOS]"] + self.tokenizer(text1)

        if text2:
            tokens += ["[SEP]"] + self.tokenizer(text2)

        tokens.append("[EOS]")

        if self.max_length:
            tokens = tokens[:self.max_length]
            tokens += ["[PAD]"] * max(0, self.max_length - len(tokens))

        return tokens

    def encode(self, text: str) -> torch.Tensor:
        """
        Encode a single text into tokens.

        Args:
            text (str): The text to encode.
        """
        tokens = self.tokenize(text)
        token_ids = [self.glove_vocab[token] for token in tokens]
        attention_mask = [0 if token_id == self.pad_idx else 1 for token_id in token_ids]
        return torch.tensor(token_ids), attention_mask

    def decode(self, token_ids: torch.Tensor) -> str:
        """
        Decode a list of token IDs into text.

        Args:
            token_ids (torch.Tensor): The list of token IDs to decode.
        """
        # Convert token IDs to tokens using the vocab's lookup_tokens method
        tokens = self.glove_vocab.lookup_tokens(token_ids.tolist())

        # Remove special tokens
        tokens = [token for token in tokens if token not in self.special_tokens]

        return " ".join(tokens)

    def encode_batch(
            self, texts: list[str]) -> tuple[list[list[int]], list[list[int]]]:
        """
        Encode a batch of texts into token IDs and attention masks.

        Args:
            texts (list[str]): The list of texts to encode.

        Returns:
            tuple: A tuple containing:
                - token_ids: List of lists of token IDs.
                - attention_masks: List of lists with 1s for real tokens and 0s for padding.
        """

        token_lists = [self.tokenize(text) for text in texts]
        token_ids = [[self.glove_vocab[token] for token in tokens]
                     for tokens in token_lists]
        attention_masks = [[0 if token_id == self.pad_idx else 1 for token_id in ids]
                           for ids in token_ids]

        return token_ids, attention_masks

    def decode_batch(self, token_ids_batch: list[torch.Tensor]) -> list[str]:
        """
        Decode a batch of token IDs into texts.

        Args:
            token_ids_batch (list[torch.Tensor]): A list of token ID tensors to decode.
        """
        # Convert each batch of token IDs to tokens using the vocab's lookup_tokens method
        texts = [
            self.glove_vocab.lookup_tokens(token_ids) for token_ids in token_ids_batch
        ]
        
        # Remove special tokens
        texts = [
            [token for token in text if token not in self.special_tokens] 
            for text in texts
        ]
        
        # Join tokens into strings for each batch
        return [" ".join(text) for text in texts]
       
    def encode_two_texts(self, context: str,
                         question: str) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Encode a single context and question into tokens.
        Args:
            context (str): The context to encode.
            question (str): The question to encode.
        """
        tokens = self.tokenize(context, question)
        token_ids = [self.glove_vocab[token] for token in tokens]
        attention_mask = [0 if token_id == self.pad_idx else 1 for token_id in token_ids]
        return torch.tensor(token_ids), attention_mask

    def encode_two_texts_batch(
            self, contexts: list[str], questions: list[str]
    ) -> tuple[list[torch.Tensor], list[list[int]]]:
        """
        Encode a batch of contexts and questions into token IDs and attention masks.

        Args:
            contexts (list[str]): The list of contexts to encode.
            questions (list[str]): The list of questions to encode.

        Returns:
            tuple: token IDs tensors list, attention masks list
        """
        token_ids = []
        attention_masks = []

        for context, question in zip(contexts, questions):
            tokens = self.tokenize(context, question)
            ids = [self.glove_vocab[token] for token in tokens]
            mask = [0 if token_id == self.pad_idx else 1 for token_id in ids]
            token_ids.append(torch.tensor(ids))
            attention_masks.append(mask)

        return token_ids, attention_masks

In [103]:
# Example usage
glove_tokenizer = gloveTokenizer()
glove_tokenizer.set_max_length(18)
text1 = "Hello, world!"
text2 = "How are you?"
encodings = glove_tokenizer.encode_two_texts(text1, text2)
print("Encoded:", encodings)
encodings_batch = glove_tokenizer.encode_batch(
    [text1, text2])
print("Encoded Batch:", encodings_batch)

encodings = glove_tokenizer.encode(text1)
print("Encoded:", encodings)
decodings = glove_tokenizer.decode(encodings[0])
print("Decoded:", decodings)

# Example usage
decoded_text = glove_tokenizer.decode_batch(
    encodings_batch[0]  # Assuming you want to decode the first batch
)
print("Decoded Batch:", decoded_text)

Encoded: (tensor([    3, 13080,     6,    90,   810,     5,   202,    37,    86,   193,
            4,     1,     1,     1,     1,     1,     1,     1]), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])
Encoded Batch: ([[3, 13080, 6, 90, 810, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 202, 37, 86, 193, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Encoded: (tensor([    3, 13080,     6,    90,   810,     4,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1]), [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Decoded: hello , world !
Decoded Batch: ['hello , world !', 'how are you ?']
